In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
week8_link = 'http://www.footballlocks.com/nfl_odds_week_8.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week8_html = requests.get(week8_link).text

soup = BeautifulSoup(week8_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[246])

<tr>
<td>10/26 8:25 ET</td>
<td>At Baltimore</td>
<td>-3</td>
<td>Miami</td>
<td>38</td>
<td>-$155 +$135</td>
</tr>
<tr>
<td>10/30 8:30 ET</td>
<td>New England</td>
<td>-2</td>
<td>At Minnesota</td>
<td>40</td>
<td>-$130 +$110</td>
</tr>


In [3]:
#want to 247 because it does not include last one
table = tr[51:247]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week8_df = pd.DataFrame(string_table)

week8_df.columns = ['all_data']

week8_df.head()

,all_data
0,<tr>\n<td>10/26 8:25 ET</td>\n<td>At Baltimore...
1,<tr>\n<td>10/29 9:30 AM</td>\n<td>Minnesota</t...
2,<tr>\n<td>10/29 1:00 ET</td>\n<td>At New Orlea...
3,<tr>\n<td>10/29 1:00 ET</td>\n<td>Atlanta</td>...
4,<tr>\n<td>10/29 1:00 ET</td>\n<td>At Tampa Bay...


In [4]:
#want to clean the date data

dates = [x.split('td')[1] for x in week8_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week8_df['date'] = cleaned_dates

In [5]:
#pull out favorites

week8_df['cleaned_favorites'] = week8_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week8_df['cleaned_underdogs'] = week8_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week8_df['cleaned_spread'] = week8_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week8_df['cleaned_over_under'] = week8_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week8_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week8_df['money_odds'] = money_odds

#input week number for each game
week8_df['week_number'] = '8'

In [6]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2017
year_list = []
for x in week8_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week8_df['year'] = year_list

In [7]:
#create a final dataframe for week 8
week8_final = week8_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week8_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,10/26 8:25 ET,2017,8,At Baltimore,Miami,-3,38,-$155 +$135
1,10/29 9:30 AM,2017,8,Minnesota,"Clevelandbrfont size=""-1""(At London)font",-11,38.5,-$600 +$450
2,10/29 1:00 ET,2017,8,At New Orleans,Chicago,-8.5,46,-$390 +$310
3,10/29 1:00 ET,2017,8,Atlanta,At NY Jets,-6.5,44,-$300 +$250
4,10/29 1:00 ET,2017,8,At Tampa Bay,Carolina,-1,46,-$120 +$100


In [8]:
#week8_final to pickle
week8_final.to_pickle('week8_final.pkl')